In [115]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Oct  1 13:11:51 2020

@author: kalyan
"""

## COMMON UTILITY FUNCTIONS - CRS Constant returns to scale. a + b = 1

import numpy as np
import sympy as sym
import plotly.graph_objects as go

########## START of iso_utility routine ##########

def iso_utility(prf,xmax=20,ymax=20,xshare=0.5,show_bline=False,price1=1,price2=1,income=100):
    
    preference = prf
    a = xshare
    b = 1-xshare
    p1 = price1
    p2 = price2
    inc = income
    show_bline = show_bline
    xmax = xmax
    ymax = ymax
    interval = 20
    x = np.linspace(0,xmax,interval,endpoint=True)
    y = np.linspace(0,ymax,interval,endpoint=True)
    x,y = np.meshgrid(x,y)
    #print(x,y)
    X, Y = sym.symbols('X Y')
    
    def get_mrs(u): # Takes a sympy function as input and returns Marginal Rate of Substitution Function.
        diff_ux = sym.diff(u,X)
        diff_uy = sym.diff(u,Y)
        return (-1)*diff_ux/diff_uy
    
    # Utility functions: return an array containing 3D Utility and 
    def cobbdouglas():
        u_func = (X**a)*(Y**b)    
        mrs_func = get_mrs(u_func)
        U = sym.lambdify((X,Y), u_func, modules = ["numpy"])
        U = U(x,y)
        MRS = sym.lambdify((X,Y), mrs_func, modules = ["numpy"])
        MRS = MRS(x,y)
        return U, MRS
    def leontief():
        c = np.vstack([x, y]) # concatenate column-wise
        u_func = np.nanmin(c, axis = 0) #find minimum value in each row.
        mrs_func = get_mrs(u_func)
        U = sym.lambdify((X,Y), u_func, modules = ["numpy"])
        U = U(x,y)
        MRS = sym.lambdify((X,Y), mrs_func, modules = ["numpy"])
        MRS = MRS(x,y)
        return U, MRS 
    def perfectsubstitute():
        u_func = (a*X) + (b*Y) #default: a=lowerbound =0.0 ; b=upperbound =1.0
        mrs_func = get_mrs(u_func)
        U = sym.lambdify((X,Y), u_func, modules = ["numpy"])
        U = U(x,y)
        MRS = [float(mrs_func) for i  in range(interval)]
        print('inside perfsub')
        return U, MRS 

    # Dispatcher aides in calling different functions based on parameters.    
    dispatcher = {'Cobb-Douglas': cobbdouglas, 
                  'Perfect-Complements': leontief, 
                  'Perfect-Substitutes': perfectsubstitute
                }

    U, MRS = dispatcher[preference]() # Assign return value of different function calls.
    
    utility_fig1 = go.Figure(data=[go.Surface(z = U)])
    utility_fig1.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))
    utility_fig1.update_layout(title=dict(
                                text='3D Utility with ({}) preferences'.format(preference),
                                font = dict(family = "Times New Roman")),
                       autosize=False, width=500, height=500,
                       scene_camera_eye=dict(x=1.87, y=0.88, z=0.64),
                               margin=dict(l=50, r=50, b=50, t=50))
    


    indiff_curve_fig2 = go.Figure(data = go.Contour(z = U, customdata = MRS, name='Indifference Curves',
                                        contours = dict(coloring='lines',showlabels = True), 
                                        line = dict(width=2, color= 'black'),
                                        hovertemplate ='<i>Utility U</i>: %{z:.3f}<br>'+
                                                  '<br><i>MRS</i>: %{customdata:.3f}<br>'+
                                                  '<br><b>X</b>: %{x}' + ' ' + '<b>Y</b>: %{y}')) 
    indiff_curve_fig2.update_layout(title=dict(
                                text='Indifference (Iso-Utility) Curves -({}) preferences'.format(preference),
                                font = dict(family = "Times New Roman")),
                                xaxis=dict(title="Good-1 (x1) units"),
                                yaxis=dict(title="Good-2 (x2) units"),
                                autosize=False, width=500, height=500,
                                margin=dict(l=50, r=50, b=50, t=50))

    ############ Budget Line Trace ############
    def get_budget_plane(p1,p2,inc,xmax,ymax,interval):

        #print(x)
        # Parameters defining the Budget Constraint
        p1 = p1
        p2 = p2
        inc = inc
        xmax = xmax
        ymax = ymax
        interval = interval
        x = np.linspace(0,xmax,interval,endpoint=True)
        y = np.linspace(0,ymax,interval,endpoint=True)
        x,y = np.meshgrid(x,y)

        X, Y = sym.symbols('X Y')

        bc_func = inc-(p1*X)+(p2*Y)   
        bc_slope = get_mrs(bc_func)
        
        BC = sym.lambdify((X,Y), bc_func, modules = ["numpy"])
        BC = BC(x,y)
        
        SLOPE = sym.lambdify((X,Y), bc_slope, modules = ["numpy"])
        SLOPE = SLOPE(x,y)
        SLOPE_LIST = [float(SLOPE) for i in range(interval)]
        #print(SLOPE_LIST)
        
        budget_surface = go.Surface(z = BC)
        utility_fig1.add_trace(budget_surface)

        trace_budgetline = go.Contour(z = BC, name='Budget line', customdata=SLOPE_LIST,  
                                      contours = dict(coloring='none',showlabels = True), 
                                        line = dict(width=1, color= 'black'),
                                      hovertemplate ='<i>Budget slope</i>: %{customdata:.3f}<br>'+
                                                      '<br><b>X</b>: %{x}' + ' ' + '<b>Y</b>: %{y}',)
    
        indiff_curve_fig2.add_trace(trace_budgetline)
        
        return utility_fig1, indiff_curve_fig2

###### Code to run if budget line is requested #######
    if show_bline == True:
        
        get_budget_plane(p1,p2,inc,xmax,ymax,interval)
        print("Budget Curve requested")

    else:
        
        print("No Budget Curve request")
        
 ###### End of coderun if budget line is requested #######
   
    return utility_fig1, indiff_curve_fig2

########## END of iso_utility routine ##########



In [117]:
f1,f2 = iso_utility('Cobb-Douglas',50,50,0.5,True,1,4,25)
f1.show()
f2.show()

Budget Curve requested


<string>:2: RuntimeWarning:

divide by zero encountered in reciprocal

<string>:2: RuntimeWarning:

invalid value encountered in multiply

